In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import metrics as m
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

### Carga de datos

In [2]:
dataset = pd.read_csv('NOSHOWS.csv')

In [3]:
dataset.head(5)

,DEPARTURE_DATE,FLIGHT_NUMBER,STD_TIME,ORIGIN,DESTINATION,TOTALNOSHOW,TotalAuthorized,TotalSeatSold,TotalSeatAvailable,WeekDayN,WeekDay,DateDay,DateMonth,DateYear,Airport ORIGIN,Country ORIGIN,Airport DESTINATION,Country DESTINATION,FESTIVO en ORIGEN,FESTIVO en DESTINO
0,"martes, 30 de abril de 2019",495,0.466667,SCL,PTY,17,154,152,2,2,Tuesday,30,4,2019,COMODORO ARTURO MERINO BENÍTEZ INTERNATIONAL A...,CHILE,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN
1,"martes, 30 de abril de 2019",276,0.700000,SCL,PTY,9,154,144,10,2,Tuesday,30,4,2019,COMODORO ARTURO MERINO BENÍTEZ INTERNATIONAL A...,CHILE,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN
2,"martes, 30 de abril de 2019",206,0.933333,ASU,PTY,2,160,153,7,2,Tuesday,30,4,2019,SILVIO PETTIROSSI INTERNATIONAL AIRPORT,PARAGUAY,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN
3,"martes, 30 de abril de 2019",758,1.333333,GRU,PTY,12,154,163,-9,2,Tuesday,30,4,2019,SÃO PAULO–GUARULHOS INTERNATIONAL AIRPORT,BRAZIL,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN
4,"martes, 30 de abril de 2019",872,1.350000,GIG,PTY,13,154,162,-8,2,Tuesday,30,4,2019,RIO DE JANEIRO–GALEÃO INTERNATIONAL AIRPORT,BRAZIL,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN


### Limpieza de la data
Se eliminan las columnas que no afectan a la prediccion

In [ ]:
#dataset.drop('FLIGHT_NUMBER', axis=1, inplace=True)
dataset.drop('DEPARTURE_DATE', axis=1, inplace=True)


Se eliminan datos en blanco o nulos

In [ ]:
dataset = dataset[dataset['TotalAuthorized'] != 0]
dataset = dataset[dataset['TotalSeatSold'] != 0]
dataset = dataset[dataset['STD_TIME'].notnull()]

Los valores de **origin** extraños se eliminan

In [ ]:
# for origin XXX XRS OOO HDQ
dataset = dataset[(dataset['ORIGIN']!='OOO') | (dataset['ORIGIN']!='XXX') | (dataset['ORIGIN']!='XRS') | (dataset['ORIGIN']!='HDQ')]

Decidimos eliminar lo *outliers* por lo que cuando la **capacidad del avion** ('TotalAuthorized') es menor que 90 y que la cantidad de personas que no se presentan <**no show**> a un vuelo particular es mayor de 60 ('TOTALNOSHOW')

In [ ]:
dataset = dataset[dataset['TotalAuthorized']>90]
dataset = dataset[dataset['TOTALNOSHOW']<60]

### Modificar las columnas utiles para la predicción
1. Se esta haciendo una transformacion numerica a _'Country Origin', 'Country Destination'_
2. Por otro lado adicionalmente de la transformacion numerica se le esta normalizando para darle mas importancia a la ciudad que al aeropuerto en cuanto a la probabilidad que se ausente un pasajero, dichas transformacion por ende se le aplicaron a _'Airport Origin', 'Airport Destination'_
3. Una vez hecho esto se procedio a crear las columnas _'Origin_Category' y 'Destination_Category'_, las cuales realmente le dan una representacion numerica a lo que es el aeropuerto y la ciudad 
4. Ahora en base a un numero decimal el programa sabe exactamente a cual aeropuerto y ciudad se refiere

In [4]:
from sklearn.preprocessing import LabelEncoder # categorizar
encoder =  LabelEncoder()
dataset['CO_Category'] = encoder.fit_transform(dataset['Country ORIGIN'])
dataset['AO_Category'] = (encoder.fit_transform(dataset['Airport ORIGIN'])/100)
dataset['Origin_Category'] = dataset['CO_Category'] + dataset['AO_Category']
dataset['CD_Category'] = encoder.fit_transform(dataset['Country DESTINATION'])
dataset['AD_Category'] = (encoder.fit_transform(dataset['Airport DESTINATION'])/100)
dataset['Destination_Category'] = dataset['CD_Category'] + dataset['AD_Category']

In [5]:
dataset.columns

Index(['DEPARTURE_DATE', 'FLIGHT_NUMBER', 'STD_TIME', 'ORIGIN', 'DESTINATION',
       'TOTALNOSHOW', 'TotalAuthorized', 'TotalSeatSold', 'TotalSeatAvailable',
       'WeekDayN', 'WeekDay', 'DateDay', 'DateMonth', 'DateYear',
       'Airport ORIGIN', 'Country ORIGIN', 'Airport DESTINATION',
       'Country DESTINATION', 'FESTIVO en ORIGEN', 'FESTIVO en DESTINO',
       'CO_Category', 'AO_Category', 'Origin_Category', 'CD_Category',
       'AD_Category', 'Destination_Category'],
      dtype='object')

In [6]:
dataset.head(5)

,DEPARTURE_DATE,FLIGHT_NUMBER,STD_TIME,ORIGIN,DESTINATION,TOTALNOSHOW,TotalAuthorized,TotalSeatSold,TotalSeatAvailable,WeekDayN,...,Airport DESTINATION,Country DESTINATION,FESTIVO en ORIGEN,FESTIVO en DESTINO,CO_Category,AO_Category,Origin_Category,CD_Category,AD_Category,Destination_Category
0,"martes, 30 de abril de 2019",495,0.466667,SCL,PTY,17,154,152,2,2,...,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN,8,0.11,8.11,24,0.82,24.82
1,"martes, 30 de abril de 2019",276,0.700000,SCL,PTY,9,154,144,10,2,...,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN,8,0.11,8.11,24,0.82,24.82
2,"martes, 30 de abril de 2019",206,0.933333,ASU,PTY,2,160,153,7,2,...,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN,25,0.74,25.74,24,0.82,24.82
3,"martes, 30 de abril de 2019",758,1.333333,GRU,PTY,12,154,163,-9,2,...,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN,6,0.76,6.76,24,0.82,24.82
4,"martes, 30 de abril de 2019",872,1.350000,GIG,PTY,13,154,162,-8,2,...,TOCUMEN INTERNATIONAL AIRPORT,PANAMA,NaN,NaN,6,0.69,6.69,24,0.82,24.82


### Creacion de un nuevo dataset
Se genera un nuevo dataset con los datos relevantes que nos ayudaran a realizar una prediccion mas precisa.

In [7]:
#Data a predecir
dataset['NOSHOW'] = dataset['TOTALNOSHOW']
dataset.drop('TOTALNOSHOW', axis=1, inplace=True)

In [8]:
#Data de entrenamiento
z = dataset[['STD_TIME', 'WeekDayN', 'DateDay', 'DateMonth', 'DateYear', 'TotalAuthorized',
             'TotalSeatSold', 'Origin_Category', 'Destination_Category']]

Es importante recalcar como los nuevos datos son plenamente numericos

In [9]:
z.head(3)

,STD_TIME,WeekDayN,DateDay,DateMonth,DateYear,TotalAuthorized,TotalSeatSold,Origin_Category,Destination_Category
0,0.466667,2,30,4,2019,154,152,8.11,24.82
1,0.700000,2,30,4,2019,154,144,8.11,24.82
2,0.933333,2,30,4,2019,160,153,25.74,24.82


## Generacion del modelo

In [11]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(z, dataset['NOSHOW'], test_size=0.3,random_state=0)

### Random Forest Regressor
Despues de realizar pruebas con diferentes modelos el que mejores resultados arrojo fue este 

In [12]:
from sklearn.ensemble import RandomForestRegressor
rand = RandomForestRegressor(random_state=0, n_estimators=64)
rand.fit(xtrain, ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
pred = rand.predict(xtest)

Comparacion de la prediccion vs de los resultados reales

In [14]:
for i in range(0, ytest.count()):
    print('%0.2f\t' % pred[i], ytest.iloc[i])

0.88	 1
0.94	 0
1.44	 2
1.31	 1
5.19	 1
4.31	 3
2.02	 4
10.17	 4
8.20	 6
5.52	 2
1.94	 1
2.59	 1
7.83	 8
0.97	 0
0.69	 0
5.09	 16
6.05	 3
8.77	 6
5.23	 2
1.09	 1
9.91	 3
0.58	 2
2.00	 0
1.56	 4
3.12	 11
4.52	 3
6.02	 3
3.62	 0
2.08	 1
1.05	 0
9.39	 2
1.47	 0
8.41	 10
3.34	 3
0.92	 0
6.31	 8
0.92	 0
4.81	 3
6.30	 3
1.09	 1
6.59	 15
5.61	 3
4.67	 5
0.67	 4
0.88	 0
0.47	 1
2.62	 2
1.92	 1
6.69	 3
0.81	 0
1.62	 0
11.81	 13
2.05	 0
0.22	 0
2.02	 1
4.73	 4
4.88	 0
14.67	 7
1.39	 2
8.42	 5
1.44	 1
3.12	 0
2.34	 0
6.02	 2
5.70	 0
2.77	 1
3.56	 5
6.44	 8
0.78	 2
1.91	 3
2.05	 8
2.00	 1
24.97	 15
3.80	 1
3.02	 0
0.95	 0
3.09	 10
3.05	 2
6.20	 10
1.92	 4
6.98	 15
6.30	 11
0.31	 0
3.30	 3
0.81	 0
1.11	 1
4.45	 2
4.39	 3
4.42	 1
0.53	 1
1.97	 1
3.62	 4
0.77	 0
12.28	 12
1.55	 0
6.83	 4
2.73	 2
6.47	 6
6.88	 5
2.33	 2
6.75	 4
0.81	 0
3.41	 7
5.75	 5
1.03	 3
7.80	 8
3.69	 0
0.98	 2
8.66	 5
1.92	 1
2.03	 3
4.30	 5
6.55	 1
1.61	 1
6.61	 9
5.27	 4
0.62	 0
0.72	 1
11.00	 17
1.66	 1
1.44	 1
6.91	 5
2.95	 

5.72	 6
3.77	 3
1.75	 3
3.59	 1
7.67	 6
7.62	 4
2.05	 0
2.45	 1
5.56	 3
2.56	 6
1.81	 3
1.95	 0
1.67	 1
1.30	 1
5.67	 13
4.23	 2
0.72	 0
0.17	 0
6.59	 15
2.61	 0
1.25	 3
7.58	 17
4.33	 7
0.66	 0
1.69	 2
1.72	 8
2.34	 3
3.44	 0
4.91	 8
9.23	 9
5.23	 2
6.50	 8
1.56	 2
5.50	 6
0.98	 0
4.28	 2
0.75	 0
6.64	 6
2.55	 0
8.84	 9
21.33	 13
0.62	 2
5.83	 13
4.72	 3
7.77	 2
1.23	 0
4.48	 4
3.22	 4
6.36	 7
2.27	 0
3.08	 3
3.62	 2
2.88	 3
7.08	 6
4.89	 4
3.02	 2
4.80	 5
3.86	 2
4.45	 1
8.14	 6
2.09	 3
3.23	 0
0.52	 5
2.19	 5
7.81	 5
8.53	 4
1.61	 4
7.38	 10
1.55	 2
1.84	 7
22.88	 27
5.45	 16
5.05	 5
3.48	 1
8.75	 7
6.62	 6
50.34	 41
1.80	 0
9.86	 3
0.86	 10
0.95	 1
2.55	 1
12.06	 12
2.12	 5
5.78	 2
0.64	 1
5.36	 4
5.00	 3
0.55	 2
4.59	 6
0.77	 2
9.06	 15
1.61	 2
3.61	 12
4.30	 0
1.20	 0
3.41	 6
9.69	 29
1.64	 0
8.86	 6
1.06	 4
11.00	 14
1.77	 3
0.45	 1
1.27	 1
3.94	 1
1.95	 0
8.62	 17
1.48	 1
6.64	 8
9.28	 8
2.28	 3
1.69	 0
0.83	 1
2.28	 1
1.50	 0
3.20	 1
3.38	 4
1.00	 0
2.17	 2
7.36	 13
4.12	 5
5.

7.97	 3
0.86	 0
4.34	 1
12.95	 2
1.80	 0
28.19	 34
5.12	 0
1.36	 0
2.02	 0
5.67	 2
5.38	 2
5.72	 7
1.38	 2
7.56	 2
2.08	 1
3.34	 1
2.00	 1
2.02	 1
4.67	 9
3.09	 7
1.36	 0
8.39	 8
4.97	 0
3.53	 1
4.05	 2
7.56	 8
26.61	 20
11.25	 30
1.53	 0
1.69	 0
2.77	 2
8.25	 2
9.06	 7
2.42	 1
3.12	 2
0.69	 1
0.77	 0
4.88	 3
1.80	 1
3.78	 2
3.48	 3
6.94	 11
6.77	 0
2.11	 3
19.22	 13
4.41	 6
10.42	 4
3.66	 1
9.02	 6
1.27	 0
4.02	 8
2.39	 2
7.58	 16
5.11	 3
6.89	 4
14.34	 6
7.25	 10
18.80	 28
1.73	 2
0.61	 0
9.62	 9
5.77	 2
5.38	 1
6.77	 5
4.61	 8
7.09	 4
5.20	 4
1.42	 2
4.81	 3
1.41	 4
6.55	 10
1.47	 0
5.42	 0
5.19	 1
1.97	 2
6.36	 6
1.55	 3
1.66	 1
10.17	 8
11.16	 7
6.05	 4
5.06	 7
6.03	 9
1.61	 1
0.64	 0
7.31	 4
53.31	 44
3.62	 5
1.27	 2
4.27	 0
0.56	 3
1.72	 3
10.66	 26
1.17	 4
1.48	 0
1.59	 0
3.52	 3
2.44	 1
0.52	 0
4.11	 8
13.72	 9
3.30	 1
0.39	 1
0.62	 0
0.61	 1
8.69	 18
6.94	 6
6.86	 4
0.50	 0
4.41	 7
1.81	 2
2.88	 0
6.22	 12
5.86	 4
1.09	 2
2.72	 8
5.66	 2
6.95	 9
1.12	 2
13.92	 14
2.31	 2
3.94

0.97	 2
1.36	 1
1.36	 1
2.48	 3
2.45	 1
6.50	 5
3.94	 1
2.70	 1
0.92	 2
1.00	 0
8.75	 3
3.66	 10
1.08	 1
2.92	 3
1.80	 3
2.78	 3
4.70	 4
7.38	 3
0.72	 3
1.47	 1
1.47	 0
1.45	 0
0.72	 2
3.56	 10
6.14	 2
5.55	 8
2.22	 1
9.14	 6
7.14	 26
1.97	 1
1.86	 4
5.59	 13
1.89	 0
6.52	 9
6.67	 4
4.89	 3
18.52	 17
0.69	 4
1.83	 3
6.89	 3
0.94	 0
10.12	 7
4.98	 2
0.80	 6
3.64	 1
1.61	 1
0.89	 0
0.27	 0
5.91	 3
2.55	 1
8.39	 10
3.91	 7
1.16	 0
10.17	 7
3.53	 2
2.14	 3
0.52	 2
0.44	 1
1.56	 1
0.94	 2
2.02	 3
4.62	 9
1.53	 0
6.50	 7
3.53	 5
0.78	 0
4.02	 2
5.48	 4
2.97	 3
2.97	 3
5.73	 6
1.19	 1
3.00	 3
4.70	 5
3.19	 4
6.97	 15
0.75	 1
2.11	 1
0.28	 0
2.73	 2
0.38	 8
3.20	 3
0.94	 2
4.64	 10
1.25	 2
3.27	 7
1.38	 0
2.03	 3
0.55	 1
3.00	 2
4.05	 2
1.80	 0
18.83	 11
0.56	 2
2.27	 7
1.28	 4
0.66	 1
1.09	 0
2.81	 2
8.39	 4
1.02	 1
2.56	 3
3.80	 3
4.48	 4
7.30	 11
12.58	 17
7.73	 5
2.72	 6
0.72	 0
1.42	 3
2.45	 0
0.94	 3
1.45	 0
8.92	 12
1.91	 1
4.33	 0
7.44	 4
0.66	 0
2.33	 0
1.58	 10
3.30	 2
9.33	 18
1.17	

1.28	 2
2.97	 3
7.70	 4
3.00	 1
9.78	 13
0.52	 0
5.41	 4
9.88	 10
0.62	 0
5.20	 6
3.33	 3
1.80	 0
9.23	 8
1.02	 4
9.53	 6
1.06	 1
1.06	 0
2.38	 1
11.98	 5
1.27	 2
4.91	 4
2.38	 3
3.45	 4
1.06	 1
5.00	 1
7.38	 6
6.83	 11
0.78	 3
3.69	 3
0.80	 0
5.81	 8
3.41	 5
3.78	 1
5.64	 5
17.47	 11
0.86	 0
1.06	 0
4.52	 3
9.56	 8
9.89	 5
3.91	 8
0.81	 1
4.89	 3
4.48	 2
1.48	 0
11.25	 6
12.16	 4
0.08	 0
6.86	 10
1.12	 0
6.17	 3
3.88	 2
6.05	 8
17.98	 7
1.73	 0
0.78	 0
3.05	 4
5.78	 12
12.05	 3
0.34	 1
1.28	 2
4.20	 2
0.72	 0
2.78	 0
7.59	 15
0.94	 0
3.77	 0
2.59	 1
2.83	 1
2.42	 0
1.14	 0
0.89	 3
0.95	 2
0.17	 13
10.50	 23
2.67	 1
6.41	 1
4.05	 1
4.16	 2
2.14	 1
4.73	 2
3.30	 4
8.00	 2
2.44	 2
5.78	 11
1.09	 0
2.77	 1
3.39	 0
4.95	 6
8.83	 17
5.61	 12
3.38	 2
2.12	 3
5.09	 2
6.56	 3
7.19	 7
2.16	 2
1.64	 3
5.52	 14
6.64	 5
10.19	 3
5.77	 5
5.83	 9
6.91	 10
5.27	 5
0.92	 0
7.62	 8
21.67	 14
0.95	 0
2.25	 2
1.62	 4
6.38	 2
3.52	 4
7.72	 2
0.75	 2
5.11	 3
8.53	 12
10.00	 8
2.64	 3
5.19	 5
4.61	 3
7.12	 

...

5.20	 1
2.14	 2
7.02	 5
1.31	 1
5.95	 10
2.03	 2
0.23	 0
3.41	 0
1.39	 2
4.91	 4
4.80	 2
3.16	 2
5.98	 5
1.03	 0
3.86	 1
2.70	 0
6.75	 3
1.05	 1
0.73	 2
3.39	 0
3.39	 0
2.30	 0
1.05	 0
6.70	 4
4.12	 2
1.84	 0
23.73	 34
3.88	 3
3.72	 3
3.94	 3
6.56	 2
8.16	 6
4.53	 4
1.20	 1
6.28	 5
5.09	 4
3.02	 1
1.47	 0
1.11	 0
0.80	 0
12.94	 10
5.38	 1
1.52	 0
2.48	 2
3.53	 7
1.94	 1
7.97	 17
9.05	 5
3.58	 5
2.69	 2
1.86	 0
2.30	 4
1.44	 0
3.48	 1
8.67	 11
5.02	 7
4.22	 2
0.39	 8
1.53	 5
11.95	 15
6.12	 12
2.16	 0
6.84	 7
2.62	 5
8.92	 10
1.23	 2
2.67	 4
16.08	 12
6.44	 3
0.61	 0
9.64	 14
19.64	 25
7.08	 6
0.91	 1
1.22	 7
10.20	 10
2.06	 4
3.23	 1
0.95	 2
6.19	 7
4.14	 3
6.91	 8
9.98	 9
0.52	 1
2.28	 2
2.17	 0
8.58	 13
7.94	 2
5.53	 3
1.19	 2
0.75	 2
5.00	 5
2.81	 2
4.25	 4
1.02	 3
1.64	 2
7.20	 7
4.91	 1
9.47	 10
1.36	 0
1.22	 3
7.31	 8
6.41	 9
0.14	 0
1.47	 1
2.28	 0
18.45	 12
3.53	 3
9.48	 9
1.03	 6
4.41	 5
7.92	 3
1.05	 0
2.12	 0
4.67	 9
2.19	 4
3.20	 0
1.20	 3
0.11	 0
1.22	 1
2.14	 0
1.88	 4
1.

0.41	 0
0.67	 1
0.80	 0
11.88	 9
1.86	 1
4.14	 4
6.00	 7
8.67	 4
7.97	 3
3.11	 6
5.36	 2
0.09	 0
6.56	 11
4.28	 3
6.33	 3
5.75	 2
4.84	 3
3.50	 10
8.42	 11
1.28	 2
2.31	 2
3.31	 2
0.70	 0
0.81	 0
0.72	 1
0.58	 0
4.39	 4
2.06	 0
1.84	 2
1.23	 0
0.53	 1
0.84	 2
4.00	 11
3.19	 1
0.89	 6
3.19	 4
9.09	 36
6.53	 7
8.78	 12
0.62	 0
0.97	 0
6.75	 2
2.86	 1
6.47	 4
4.73	 6
5.67	 13
5.34	 3
7.38	 5
0.22	 0
4.72	 2
0.38	 0
5.83	 2
8.61	 5
4.53	 5
2.16	 0
1.42	 1
2.30	 3
0.23	 1
3.39	 1
0.88	 1
10.19	 4
3.14	 6
10.58	 7
2.91	 1
1.94	 2
3.09	 9
1.33	 0
0.30	 0
1.77	 0
1.17	 3
6.91	 1
6.27	 1
1.59	 1
1.23	 0
1.52	 4
4.27	 8
2.94	 8
1.05	 2
0.94	 1
1.09	 3
3.94	 5
2.17	 1
1.45	 1
4.38	 7
0.19	 0
1.42	 0
3.25	 1
2.64	 5
5.22	 3
1.17	 1
21.12	 31
6.77	 2
0.94	 1
8.69	 10
5.00	 7
1.31	 3
4.47	 12
2.03	 2
4.98	 2
0.91	 0
1.89	 6
7.39	 11
2.59	 1
4.55	 1
1.03	 0
0.75	 1
0.66	 2
0.28	 0
5.92	 4
4.44	 6
2.92	 5
7.03	 5
1.45	 1
1.84	 1
0.91	 1
7.55	 4
1.20	 2
2.28	 4
1.73	 3
0.92	 2
5.73	 5
3.42	 3
3.56	 3
8

11.75	 7
0.98	 1
0.30	 0
6.84	 3
10.11	 14
6.44	 5
1.09	 1
5.83	 4
2.95	 5
1.56	 1
4.59	 1
6.48	 5
1.38	 0
1.03	 0
1.02	 0
0.73	 2
6.67	 7
2.31	 1
2.75	 5
3.88	 10
10.91	 3
6.81	 8
5.50	 2
1.62	 1
6.14	 5
15.09	 21
0.97	 1
8.41	 9
1.98	 1
1.33	 1
18.00	 15
5.38	 2
7.67	 5
2.06	 1
5.45	 5
3.59	 5
17.22	 9
6.09	 6
7.05	 5
1.56	 2
7.22	 6
11.70	 8
2.28	 2
10.20	 14
8.30	 5
1.98	 0
2.08	 1
3.27	 2
2.67	 0
6.34	 7
6.34	 5
3.61	 6
5.62	 6
4.19	 11
11.69	 4
7.61	 7
15.62	 15
1.17	 4
3.41	 5
2.66	 4
7.52	 2
1.72	 0
0.78	 0
1.66	 1
1.50	 1
6.23	 2
4.41	 1
5.22	 9
11.80	 18
4.86	 1
11.62	 7
0.28	 0
5.64	 10
5.38	 3
3.56	 1
1.80	 4
1.69	 3
1.02	 3
6.98	 1
3.77	 4
2.88	 3
2.84	 0
5.83	 2
10.72	 8
1.03	 0
4.84	 2
2.97	 3
5.48	 4
0.80	 3
1.41	 1
1.72	 3
3.09	 1
2.06	 1
1.81	 4
0.66	 0
9.22	 13
0.83	 0
6.34	 3
3.11	 2
4.95	 5
4.03	 1
10.77	 8
2.50	 1
0.94	 3
2.03	 0
6.31	 21
5.61	 4
1.20	 0
4.94	 3
8.88	 6
4.52	 2
0.66	 0
3.28	 2
11.62	 9
9.53	 7
3.84	 1
4.02	 0
3.66	 8
2.00	 1
0.39	 0
1.58	 3
1.20	 

8.73	 9
5.09	 0
3.52	 5
4.27	 1
6.47	 5
2.09	 0
0.88	 1
9.33	 12
1.58	 0
1.95	 0
11.12	 3
0.52	 1
1.19	 1
4.89	 2
6.75	 3
7.06	 3
14.66	 24
12.69	 7
10.50	 34
1.00	 4
8.02	 4
2.12	 0
4.38	 3
0.92	 3
8.86	 16
0.86	 0
6.00	 5
1.20	 3
1.83	 0
2.53	 1
2.22	 2
1.58	 0
2.11	 0
3.47	 6
1.98	 2
3.16	 0
2.09	 0
8.81	 1
0.78	 0
1.86	 0
4.39	 3
5.70	 35
0.77	 0
1.11	 1
3.47	 8
1.78	 1
2.66	 9
9.33	 7
0.47	 0
4.80	 5
1.00	 0
4.05	 4
2.03	 0
5.97	 1
1.61	 4
7.61	 7
28.64	 31
0.83	 0
7.67	 8
1.45	 2
1.34	 0
5.70	 8
1.34	 1
7.28	 4
2.38	 1
2.12	 1
1.31	 2
6.91	 6
1.42	 3
2.42	 0
2.06	 1
3.78	 0
5.05	 5
6.12	 10
1.67	 0
5.25	 6
9.70	 7
0.45	 0
1.06	 0
7.22	 9
5.08	 0
3.31	 1
11.78	 12
4.00	 5
1.52	 1
10.38	 9
1.03	 1
1.98	 0
0.22	 0
2.19	 0
3.22	 0
5.80	 2
1.50	 2
3.14	 2
1.53	 0
1.50	 0
3.50	 1
1.73	 0
0.70	 1
1.19	 1
4.67	 6
4.62	 18
4.89	 5
0.45	 0
1.31	 0
1.94	 5
0.80	 1
0.22	 0
0.88	 1
4.52	 3
20.12	 10
0.44	 0
12.00	 19
2.97	 0
61.92	 57
1.41	 0
0.72	 9
2.48	 0
9.52	 9
9.81	 9
7.16	 10
0.45	 0
1

4.05	 5
1.75	 3
1.70	 0
3.34	 1
2.11	 2
0.80	 0
7.16	 4
4.73	 4
9.00	 35
0.64	 2
6.12	 10
5.00	 11
21.98	 14
7.78	 10
1.64	 0
7.25	 8
5.70	 6
3.70	 6
9.55	 15
2.12	 0
6.08	 6
8.42	 7
16.14	 15
0.33	 2
3.84	 4
24.59	 15
4.23	 3
1.05	 1
5.77	 8
3.88	 0
1.17	 0
1.30	 0
0.36	 0
6.69	 8
1.56	 0
1.34	 1
0.17	 1
3.67	 2
1.05	 0
2.59	 0
6.88	 6
1.48	 2
5.02	 0
1.00	 0
1.89	 1
2.31	 0
5.12	 6
5.22	 4
2.42	 13
20.14	 20
1.27	 1
4.97	 2
0.81	 0
6.56	 8
5.48	 1
2.83	 4
0.95	 4
1.55	 1
2.38	 0
1.28	 5
1.19	 2
8.02	 6
2.55	 1
1.38	 4
12.36	 6
6.69	 6
0.94	 1
3.80	 2
4.55	 9
3.33	 5
3.50	 1
2.14	 1
2.25	 0
3.34	 1
3.30	 0
1.56	 4
1.98	 1
5.12	 6
10.31	 12
4.27	 2
5.89	 3
1.12	 0
7.94	 8
2.02	 1
1.75	 2
4.33	 11
8.75	 2
8.48	 9
1.62	 7
6.50	 6
15.75	 7
5.22	 3
1.17	 0
34.97	 38
1.84	 1
0.92	 1
3.55	 1
0.50	 1
1.28	 0
4.81	 4
0.95	 0
4.75	 3
5.30	 2
6.50	 3
6.11	 2
0.98	 0
15.09	 12
11.39	 13
4.92	 7
2.55	 2
4.33	 8
4.72	 3
1.58	 1
2.14	 5
3.28	 4
1.86	 3
2.44	 1
1.52	 2
5.19	 5
5.95	 5
30.66	 27
16.81

In [15]:
absol = m.mean_absolute_error(ytest, pred)
print("Mean abs: \t %0.5f" % absol)
print("Mean Square: \t %0.5f" % (m.mean_squared_error(ytest, pred)))
print("Variance: \t %0.5f" % (m.r2_score(ytest, pred)))

Mean abs: 	 2.25910
Mean Square: 	 11.65082
Variance: 	 0.59681


### Recolección de los datos de entrada
Datos de entrada que se le piden al usuario

In [16]:
# tiempo, dia de semana, dia, mes, año, capacidad, vendidos, origen, destino

tiempo = float(input("Hora de salida: ")) # Tiempo de vuelo en flotante
weekday = float(input("Dia de semana: ")) # en numero 
dateday = float(input("Fecha dia: ")) # en numero
datemonth = float(input("Fecha mes: ")) # en numero
dateyear = float(input("Fecha año: "))
capac = float(input("Capacidad: "))
vend = float(input("Vendidos: "))
ori = input("Pais de origen: ") # Mayusculas
dest = input("Pais de destino: ") # mayusculas

Hora de salida: 6.0
Dia de semana: 7
Fecha dia: 21
Fecha mes: 11
Fecha año: 2020
Capacidad: 200
Vendidos: 230
Pais de origen: CHILE
Pais de destino: COLOMBIA


Se filtran los datos segun el pais de origen y destino mencionados 

In [17]:
c_origin = dataset[['Country ORIGIN', 'CO_Category']].drop_duplicates(['Country ORIGIN', 'CO_Category'])
cod_origin = c_origin['CO_Category'].loc[c_origin['Country ORIGIN']==ori].values[0]

In [18]:
c_dest = dataset[['Country DESTINATION', 'CD_Category']].drop_duplicates(['Country DESTINATION', 'CD_Category'])
cod_dest = c_dest['CD_Category'].loc[c_dest['Country DESTINATION']==dest].values[0]

Se crea un arreglo con los datos de entrada para luego introducirlos al modelo predictivo

In [19]:
data = np.array([[tiempo, weekday, dateday, datemonth, dateyear,
        capac, vend, cod_origin, cod_dest]])

In [20]:
prediction = int(rand.predict(data))

### Predicción final

In [21]:
print('NOSHOW DE: ', prediction, ' persona(s)')

NOSHOW DE:  8  persona(s)
